## Environment Settings

In [1]:
%load_ext dotenv
%dotenv
from pdnd.common.functions import Session
from pdnd.common.google_utils import Sheet
from pdnd.common.functions import *

In [2]:
session = Session.from_env()

In [3]:
pdq = session.get_panda_query('impala')

## Create Sheet instance

In [4]:
# sheet representation provide useful information
example_sheet = Sheet(sheet_id='1Ts6LJ0P4kt-K7t5k6HF4YxhrzvYoWrsAKvOTRdCXD1o' , id_mode='key')
example_sheet

Google sheet (id:'1Ts6LJ0P4kt-K7t5k6HF4YxhrzvYoWrsAKvOTRdCXD1o'|title:'Prova'|id_mode:'key'|url: https://docs.google.com/spreadsheets/d/1Ts6LJ0P4kt-K7t5k6HF4YxhrzvYoWrsAKvOTRdCXD1o)

## Query

In [5]:
report = pdq(f'''SELECT n.iun, n.sentat, n.senderpaid, n.senderdenomination, n.physicalCommunicationType, n.subject, n.taxonomycode, get_json_object(t.details, '$.physicalAddress.zip') cap, CAST(get_json_object(t.details, '$.numberOfPages') as bigint) pages, CAST(get_json_object(t.details, '$.envelopeWeight') as bigint) weight
            FROM send.silver_timeline t
            join send.silver_notification n on n.iun = t.iun
            left OUTER join send.silver_timeline cancelled on (t.iun = cancelled.iun and t.paid = cancelled.paid and cancelled.category ='NOTIFICATION_CANCELLED')
            left OUTER join send.silver_timeline viewed on (t.iun = viewed.iun and t.paid = viewed.paid and viewed.category = 'NOTIFICATION_VIEWED')
            WHERE t.category='SEND_ANALOG_DOMICILE'
            and cancelled.iun is null 
            and viewed.iun is null
            and ((CAST(get_json_object(t.details, '$.numberOfPages') as bigint) > 18 and n.physicalcommunicationtype = 'REGISTERED_LETTER_890') or 
            (CAST(get_json_object(t.details, '$.numberOfPages') as bigint) > 100 and n.physicalcommunicationtype = 'AR_REGISTERED_LETTER'))
            ''')

## Uploading

In [6]:
example_sheet.upload('notifiche_bloccate', report)